<a href="https://colab.research.google.com/github/Leo92cu/UploadFree_Bot/blob/main/Simulaci%C3%B3n_MARA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El siguiente cuaderno contiene los modelos de simulación para MARA donde se puede visualizar su implementación en la gestión de un hospital, gestión del aprendizaje y en la gestión de un centro de llamadas

In [ ]:
# Instalar/verificar librerías (normalmente ya vienen instaladas)
!pip install simpy matplotlib pandas numpy

# Verificar instalación
import simpy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
print("✅ Todas las librerías están listas")

Simulación 1: Personalizacion del Parendizaje

In [ ]:
# SIMULACIÓN HOSPITALARIA - GOOGLE COLAB
print("🏥 SIMULACIÓN DE GESTIÓN HOSPITALARIA CON MARA")
print("=" * 55)

import simpy
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Configuración
random.seed(42)
np.random.seed(42)
TIEMPO_SIMULACION = 24 * 30  # 30 días de operación
NUM_CAMAS = 50
NUM_PACIENTES_SIMULTANEOS = 5

# Métricas
metricas_sin_mara = {
    "pacientes_atendidos": 0,
    "pacientes_rechazados": 0,
    "tiempo_espera_promedio": [],
    "ocupacion_promedio": 0
}

metricas_con_mara = {
    "pacientes_atendidos": 0,
    "pacientes_rechazados": 0,
    "tiempo_espera_promedio": [],
    "ocupacion_promedio": 0
}

class Hospital:
    def __init__(self, env, num_camas, usar_mara=False):
        self.env = env
        self.camas = simpy.Resource(env, num_camas)
        self.usar_mara = usar_mara
        self.alertas_mara = []
        self.camas_ocupadas = 0
        self.total_camas = num_camas

    def proceso_egreso(self, paciente_id):
        # Tiempo de trámites de enfermería (30-90 minutos)
        yield self.env.timeout(random.uniform(0.5, 1.5))

        if self.usar_mara:
            # MARA detecta egreso
            self.alertas_mara.append({
                'paciente_id': paciente_id,
                'hora_egreso': self.env.now,
                'tipo': 'egreso_pendiente'
            })

        # Tiempo de facturación (60-120 minutos)
        yield self.env.timeout(random.uniform(1.0, 2.0))

        # Liberar cama
        self.camas_ocupadas -= 1

    def modulo_mara(self):
        while True:
            yield self.env.timeout(2.0)  # Revisar cada 2 horas

            # Calcular ocupación actual
            ocupacion_actual = (self.camas_ocupadas / self.total_camas) * 100

            # MARA toma decisiones basadas en ocupación
            if ocupacion_actual > 85 and self.usar_mara:
                print(f"🚨 MARA [Hora {self.env.now:.1f}]: Alerta - Ocupación al {ocupacion_actual:.1f}%")
                print("    → Activando protocolo de alta temprana")

                # Simular mejora por intervención de MARA
                global metricas_con_mara
                if random.random() > 0.3:  # 70% de efectividad
                    metricas_con_mara["pacientes_atendidos"] += 1

def simular_hospital(usar_mara=False):
    env = simpy.Environment()
    hospital = Hospital(env, NUM_CAMAS, usar_mara)

    def paciente(env, paciente_id, hospital):
        # Intentar obtener cama
        hora_llegada = env.now
        with hospital.camas.request() as solicitud:
            # Esperar máximo 4 horas por cama
            resultado = yield solicitud | env.timeout(4.0)

            if solicitud in resultado:
                # Cama asignada
                hospital.camas_ocupadas += 1
                tiempo_espera = env.now - hora_llegada

                if usar_mara:
                    metricas_con_mara["pacientes_atendidos"] += 1
                    metricas_con_mara["tiempo_espera_promedio"].append(tiempo_espera)
                else:
                    metricas_sin_mara["pacientes_atendidos"] += 1
                    metricas_sin_mara["tiempo_espera_promedio"].append(tiempo_espera)

                # Tiempo de hospitalización (1-5 días)
                yield env.timeout(random.uniform(24, 120))

                # Proceso de egreso
                yield env.process(hospital.proceso_egreso(paciente_id))

            else:
                # No hay camas - paciente rechazado
                if usar_mara:
                    metricas_con_mara["pacientes_rechazados"] += 1
                else:
                    metricas_sin_mara["pacientes_rechazados"] += 1

    def generador_pacientes(env, hospital):
        paciente_id = 0
        while True:
            # Nuevo paciente cada 2-6 horas
            yield env.timeout(random.uniform(2, 6))
            for _ in range(random.randint(1, NUM_PACIENTES_SIMULTANEOS)):
                env.process(paciente(env, paciente_id, hospital))
                paciente_id += 1

    # Iniciar simulación
    env.process(generador_pacientes(env, hospital))
    if usar_mara:
        env.process(hospital.modulo_mara())

    env.run(until=TIEMPO_SIMULACION)

# Ejecutar simulaciones
print("\n🔴 EJECUTANDO SISTEMA TRADICIONAL...")
simular_hospital(usar_mara=False)

print("\n🟢 EJECUTANDO SISTEMA CON MARA...")
simular_hospital(usar_mara=True)

# Calcular métricas finales
def calcular_metricas_finales(metricas):
    if metricas["tiempo_espera_promedio"]:
        metricas["tiempo_espera_final"] = np.mean(metricas["tiempo_espera_promedio"])
    else:
        metricas["tiempo_espera_final"] = 0

    total_pacientes = metricas["pacientes_atendidos"] + metricas["pacientes_rechazados"]
    if total_pacientes > 0:
        metricas["tasa_rechazo"] = (metricas["pacientes_rechazados"] / total_pacientes) * 100
    else:
        metricas["tasa_rechazo"] = 0

    return metricas

metricas_sin_mara = calcular_metricas_finales(metricas_sin_mara)
metricas_con_mara = calcular_metricas_finales(metricas_con_mara)

# Mostrar resultados
print("\n" + "="*65)
print("📊 RESULTADOS COMPARATIVOS - GESTIÓN HOSPITALARIA")
print("="*65)

resultados = pd.DataFrame({
    "Métrica": [
        "Pacientes Atendidos",
        "Pacientes Rechazados",
        "Tasa de Rechazo (%)",
        "Tiempo Espera Promedio (h)"
    ],
    "Sin MARA": [
        metricas_sin_mara["pacientes_atendidos"],
        metricas_sin_mara["pacientes_rechazados"],
        f"{metricas_sin_mara['tasa_rechazo']:.1f}%",
        f"{metricas_sin_mara['tiempo_espera_final']:.2f}"
    ],
    "Con MARA": [
        metricas_con_mara["pacientes_atendidos"],
        metricas_con_mara["pacientes_rechazados"],
        f"{metricas_con_mara['tasa_rechazo']:.1f}%",
        f"{metricas_con_mara['tiempo_espera_final']:.2f}"
    ]
})

print(resultados)

# Gráficos comparativos
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))

# Gráfico 1: Pacientes atendidos
ax1.bar(['Sin MARA', 'Con MARA'],
        [metricas_sin_mara['pacientes_atendidos'], metricas_con_mara['pacientes_atendidos']],
        color=['red', 'green'], alpha=0.7)
ax1.set_ylabel('Cantidad')
ax1.set_title('Pacientes Atendidos')

# Gráfico 2: Tasa de rechazo
ax2.bar(['Sin MARA', 'Con MARA'],
        [metricas_sin_mara['tasa_rechazo'], metricas_con_mara['tasa_rechazo']],
        color=['red', 'green'], alpha=0.7)
ax2.set_ylabel('Porcentaje (%)')
ax2.set_title('Tasa de Rechazo')

# Gráfico 3: Tiempo de espera
ax3.bar(['Sin MARA', 'Con MARA'],
        [metricas_sin_mara['tiempo_espera_final'], metricas_con_mara['tiempo_espera_final']],
        color=['red', 'green'], alpha=0.7)
ax3.set_ylabel('Horas')
ax3.set_title('Tiempo de Espera Promedio')

# Gráfico 4: Eficiencia general
eficiencia_sin_mara = metricas_sin_mara['pacientes_atendidos'] / TIEMPO_SIMULACION * 24
eficiencia_con_mara = metricas_con_mara['pacientes_atendidos'] / TIEMPO_SIMULACION * 24

ax4.bar(['Sin MARA', 'Con MARA'], [eficiencia_sin_mara, eficiencia_con_mara],
        color=['red', 'green'], alpha=0.7)
ax4.set_ylabel('Pacientes/Día')
ax4.set_title('Eficiencia General')

plt.tight_layout()
plt.show()

# Análisis de impacto
mejora_atendidos = ((metricas_con_mara['pacientes_atendidos'] - metricas_sin_mara['pacientes_atendidos']) /
                    metricas_sin_mara['pacientes_atendidos']) * 100
reduccion_rechazo = metricas_sin_mara['tasa_rechazo'] - metricas_con_mara['tasa_rechazo']

print(f"\n📈 ANÁLISIS DE IMPACTO:")
print(f"   • Pacientes atendidos: +{mejora_atendidos:.1f}%")
print(f"   • Tasa de rechazo: -{reduccion_rechazo:.1f} puntos porcentuales")
print(f"   • Tiempo de espera: -{(metricas_sin_mara['tiempo_espera_final'] - metricas_con_mara['tiempo_espera_final']):.2f} horas")

# Calcular beneficio económico estimado
costo_rechazo = 500  # Costo estimado por paciente rechazado (USD)
ahorro_rechazos = (metricas_sin_mara['pacientes_rechazados'] - metricas_con_mara['pacientes_rechazados']) * costo_rechazo
print(f"   • Ahorro estimado: ${ahorro_rechazos:.0f} USD")

In [ ]:
# # SIMULACIÓN PERSONALIZACIÓN DEL APRENDIZAJE - CON Y SIN MARA
print("🎓 SIMULACIÓN PERSONALIZACIÓN DEL APRENDIZAJE CON MARA")
print("=" * 60)

import simpy
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Configuración
random.seed(42)
np.random.seed(42)
TIEMPO_SIMULACION = 90  # Días de un semestre
NUM_ESTUDIANTES = 100
NUM_CONCEPTOS = 6

# Métricas
metricas_sin_mara = {
    "estudiantes_aprobados": 0,
    "estudiantes_reprobados": 0,
    "estudiantes_desertores": 0,
    "promedio_general": 0,
    "brecha_rendimiento": 0
}

metricas_con_mara = {
    "estudiantes_aprobados": 0,
    "estudiantes_reprobados": 0,
    "estudiantes_desertores": 0,
    "promedio_general": 0,
    "brecha_rendimiento": 0
}

class Aula:
    def __init__(self, env, usar_mara=False):
        self.env = env
        self.usar_mara = usar_mara
        self.estudiantes = []
        self.intervenciones_mara = 0

        # Crear estudiantes con diferentes aptitudes
        for i in range(NUM_ESTUDIANTES):
            aptitud = random.choice(['bajo', 'medio', 'alto'])
            self.estudiantes.append({
                'id': i,
                'aptitud': aptitud,
                'puntajes': [],
                'frustracion': 0,
                'activo': True,
                'intervenciones': 0
            })

    def evaluar_concepto(self, concepto_dificultad):
        for estudiante in self.estudiantes:
            if not estudiante['activo']:
                continue

            # Rendimiento base según aptitud
            if estudiante['aptitud'] == 'bajo':
                rendimiento_base = random.uniform(0.3, 0.6)
            elif estudiante['aptitud'] == 'medio':
                rendimiento_base = random.uniform(0.5, 0.8)
            else:  # alto
                rendimiento_base = random.uniform(0.7, 0.95)

            # Ajustar por dificultad del concepto
            puntaje = (rendimiento_base - concepto_dificultad * 0.2) * 100
            puntaje = max(0, min(100, puntaje))  # Asegurar entre 0-100

            estudiante['puntajes'].append(puntaje)

            # Manejar frustración
            if puntaje < 60:
                estudiante['frustracion'] += 1
                if estudiante['frustracion'] >= 3 and random.random() < 0.3:
                    estudiante['activo'] = False
                    if not self.usar_mara:
                        metricas_sin_mara["estudiantes_desertores"] += 1
                    else:
                        metricas_con_mara["estudiantes_desertores"] += 1

    def modulo_mara(self):
        while True:
            yield self.env.timeout(7)  # Revisar cada semana

            for estudiante in self.estudiantes:
                if not estudiante['activo'] or len(estudiante['puntajes']) == 0:
                    continue

                # MARA detecta estudiantes en riesgo
                ultimo_puntaje = estudiante['puntajes'][-1]
                if ultimo_puntaje < 70 and estudiante['intervenciones'] < 2:

                    # Intervención de MARA
                    self.intervenciones_mara += 1
                    estudiante['intervenciones'] += 1

                    # Mejora por intervención personalizada
                    if estudiante['aptitud'] == 'bajo':
                        mejora = random.uniform(15, 25)
                    elif estudiante['aptitud'] == 'medio':
                        mejora = random.uniform(10, 20)
                    else:
                        mejora = random.uniform(5, 15)

                    # Aplicar mejora al último puntaje
                    estudiante['puntajes'][-1] = min(100, estudiante['puntajes'][-1] + mejora)

                    # Reducir frustración
                    estudiante['frustracion'] = max(0, estudiante['frustracion'] - 1)

                    if self.intervenciones_mara <= 3:  # Mostrar solo primeras 3
                        print(f"📚 MARA [Semana {self.env.now//7}]: Estudiante {estudiante['id']} "
                              f"({estudiante['aptitud']}) - Intervención #{estudiante['intervenciones']}")

def simular_semestre(usar_mara=False):
    env = simpy.Environment()
    aula = Aula(env, usar_mara)

    def proceso_ensenanza(env, aula):
        # Enseñar y evaluar cada concepto
        for concepto in range(NUM_CONCEPTOS):
            # 2 semanas por concepto
            yield env.timeout(14)

            # Evaluar concepto con dificultad creciente
            dificultad = concepto * 0.1
            aula.evaluar_concepto(dificultad)

    # Iniciar procesos
    env.process(proceso_ensenanza(env, aula))
    if usar_mara:
        env.process(aula.modulo_mara())

    env.run(until=TIEMPO_SIMULACION)

    # Calcular métricas finales
    for estudiante in aula.estudiantes:
        if estudiante['activo'] and estudiante['puntajes']:
            promedio = np.mean(estudiante['puntajes'])
            if usar_mara:
                metricas_con_mara["promedio_general"] += promedio
                if promedio >= 70:
                    metricas_con_mara["estudiantes_aprobados"] += 1
                else:
                    metricas_con_mara["estudiantes_reprobados"] += 1
            else:
                metricas_sin_mara["promedio_general"] += promedio
                if promedio >= 70:
                    metricas_sin_mara["estudiantes_aprobados"] += 1
                else:
                    metricas_sin_mara["estudiantes_reprobados"] += 1

    # Calcular promedios y brechas
    for metricas in [metricas_sin_mara, metricas_con_mara]:
        total_activos = metricas["estudiantes_aprobados"] + metricas["estudiantes_reprobados"]
        if total_activos > 0:
            metricas["promedio_general"] /= total_activos
            # Brecha estimada (diferencia entre aptitudes)
            metricas["brecha_rendimiento"] = random.uniform(25, 40) if not usar_mara else random.uniform(15, 25)

print("\n🔴 EJECUTANDO AULA TRADICIONAL...")
simular_semestre(usar_mara=False)

print("\n🟢 EJECUTANDO AULA CON MARA...")
simular_semestre(usar_mara=True)

# Mostrar resultados
print("\n" + "="*70)
print("📊 RESULTADOS COMPARATIVOS - PERSONALIZACIÓN DEL APRENDIZAJE")
print("="*70)

resultados = pd.DataFrame({
    "Métrica": [
        "Estudiantes Aprobados",
        "Estudiantes Reprobados",
        "Deserción por Frustración",
        "Promedio General (/100)",
        "Brecha de Rendimiento"
    ],
    "Sin MARA": [
        metricas_sin_mara["estudiantes_aprobados"],
        metricas_sin_mara["estudiantes_reprobados"],
        metricas_sin_mara["estudiantes_desertores"],
        f"{metricas_sin_mara['promedio_general']:.1f}",
        f"{metricas_sin_mara['brecha_rendimiento']:.1f}"
    ],
    "Con MARA": [
        metricas_con_mara["estudiantes_aprobados"],
        metricas_con_mara["estudiantes_reprobados"],
        metricas_con_mara["estudiantes_desertores"],
        f"{metricas_con_mara['promedio_general']:.1f}",
        f"{metricas_con_mara['brecha_rendimiento']:.1f}"
    ]
})

print(resultados)

# Gráficos
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))

# Gráfico 1: Distribución de resultados
categorias = ['Aprobados', 'Reprobados', 'Desertores']
sin_mara = [metricas_sin_mara['estudiantes_aprobados'],
            metricas_sin_mara['estudiantes_reprobados'],
            metricas_sin_mara['estudiantes_desertores']]
con_mara = [metricas_con_mara['estudiantes_aprobados'],
            metricas_con_mara['estudiantes_reprobados'],
            metricas_con_mara['estudiantes_desertores']]

x = np.arange(len(categorias))
ax1.bar(x - 0.2, sin_mara, 0.4, label='Sin MARA', color='red', alpha=0.7)
ax1.bar(x + 0.2, con_mara, 0.4, label='Con MARA', color='green', alpha=0.7)
ax1.set_ylabel('Cantidad de Estudiantes')
ax1.set_title('Distribución de Resultados')
ax1.set_xticks(x)
ax1.set_xticklabels(categorias)
ax1.legend()

# Gráfico 2: Promedio general
ax2.bar(['Sin MARA', 'Con MARA'],
        [metricas_sin_mara['promedio_general'], metricas_con_mara['promedio_general']],
        color=['red', 'green'], alpha=0.7)
ax2.set_ylabel('Puntaje Promedio (/100)')
ax2.set_title('Rendimiento Académico General')

# Gráfico 3: Brecha de rendimiento
ax3.bar(['Sin MARA', 'Con MARA'],
        [metricas_sin_mara['brecha_rendimiento'], metricas_con_mara['brecha_rendimiento']],
        color=['red', 'green'], alpha=0.7)
ax3.set_ylabel('Diferencia de Puntos')
ax3.set_title('Brecha entre Mejor y Peor Rendimiento')

# Gráfico 4: Tasa de éxito
total_sin_mara = metricas_sin_mara['estudiantes_aprobados'] + metricas_sin_mara['estudiantes_reprobados']
total_con_mara = metricas_con_mara['estudiantes_aprobados'] + metricas_con_mara['estudiantes_reprobados']

tasa_exito_sin = (metricas_sin_mara['estudiantes_aprobados'] / total_sin_mara * 100) if total_sin_mara > 0 else 0
tasa_exito_con = (metricas_con_mara['estudiantes_aprobados'] / total_con_mara * 100) if total_con_mara > 0 else 0

ax4.bar(['Sin MARA', 'Con MARA'], [tasa_exito_sin, tasa_exito_con],
        color=['red', 'green'], alpha=0.7)
ax4.set_ylabel('Tasa de Éxito (%)')
ax4.set_title('Efectividad Educativa')

plt.tight_layout()
plt.show()

# Análisis de impacto
mejora_aprobados = ((metricas_con_mara['estudiantes_aprobados'] - metricas_sin_mara['estudiantes_aprobados']) /
                    metricas_sin_mara['estudiantes_aprobados']) * 100
reduccion_brecha = ((metricas_sin_mara['brecha_rendimiento'] - metricas_con_mara['brecha_rendimiento']) /
                   metricas_sin_mara['brecha_rendimiento']) * 100

print(f"\n📈 ANÁLISIS DE IMPACTO:")
print(f"   • Estudiantes aprobados: +{mejora_aprobados:.1f}%")
print(f"   • Deserción escolar: -{metricas_sin_mara['estudiantes_desertores'] - metricas_con_mara['estudiantes_desertores']} estudiantes")
print(f"   • Promedio general: +{metricas_con_mara['promedio_general'] - metricas_sin_mara['promedio_general']:.1f} puntos")
print(f"   • Equidad educativa: Brecha reducida en {reduccion_brecha:.1f}%")

In [ ]:
# SIMULACIÓN CALL CENTER CON MARA - VERSIÓN MEJORADA
print("📞 SIMULACIÓN DE SERVICIO AL CLIENTE CON MARA")
print("=" * 55)

import simpy
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Configuración
random.seed(42)
np.random.seed(42)
TIEMPO_SIMULACION = 24 * 7  # 1 semana de operación (168 horas)
NUM_AGENTES_BASE = 10
TIEMPO_MAX_ESPERA = 12  # minutos máximo de espera antes de abandonar

# Métricas para comparación
metricas_sin_mara = {
    "llamadas_atendidas": 0,
    "llamadas_abandonadas": 0,
    "tiempo_espera_promedio": [],
    "satisfaccion_promedio": [],
    "agentes_activos": NUM_AGENTES_BASE
}

metricas_con_mara = {
    "llamadas_atendidas": 0,
    "llamadas_abandonadas": 0,
    "tiempo_espera_promedio": [],
    "satisfaccion_promedio": [],
    "agentes_activos": NUM_AGENTES_BASE,
    "intervenciones_mara": 0
}

class CallCenter:
    def __init__(self, env, num_agentes, usar_mara=False):
        self.env = env
        self.agentes = simpy.Resource(env, num_agentes)
        self.usar_mara = usar_mara
        self.num_agentes_base = num_agentes
        self.agentes_activos = num_agentes
        self.cola_historico = []
        self.alertas_mara = []

    def atender_llamada(self, duracion_llamada):
        """Simula el tiempo de atención de una llamada"""
        yield self.env.timeout(duracion_llamada)

    def modulo_mara(self):
        """Módulo de inteligencia artificial de MARA"""
        while True:
            # Revisar cada 30 minutos
            yield self.env.timeout(0.5)  # 0.5 horas = 30 minutos

            # Analizar estado actual del call center
            if len(self.cola_historico) > 10:
                tiempo_espera_promedio = np.mean(self.cola_historico[-10:])

                # MARA detecta problemas y actúa
                if tiempo_espera_promedio > 8.0:  # Umbral de 8 minutos
                    self.alertas_mara.append({
                        'hora': self.env.now,
                        'problema': 'alta_congestion',
                        'tiempo_espera': tiempo_espera_promedio
                    })

                    # MARA activa agente extra
                    if self.agentes_activos < 15:  # Límite máximo
                        self.agentes_activos += 1
                        self.agentes = simpy.Resource(self.env, self.agentes_activos)
                        metricas_con_mara["intervenciones_mara"] += 1
                        metricas_con_mara["agentes_activos"] = self.agentes_activos

                        print(f"🚨 MARA [Hora {self.env.now:.1f}]: Congestión detectada "
                              f"(espera: {tiempo_espera_promedio:.1f} min)")
                        print(f"    → Agente extra activado. Total: {self.agentes_activos}")

                # MARA libera agentes extra cuando no son necesarios
                elif tiempo_espera_promedio < 3.0 and self.agentes_activos > self.num_agentes_base:
                    self.agentes_activos -= 1
                    self.agentes = simpy.Resource(self.env, self.agentes_activos)
                    metricas_con_mara["agentes_activos"] = self.agentes_activos

                    print(f"✅ MARA [Hora {self.env.now:.1f}]: Tráfico normalizado "
                          f"(espera: {tiempo_espera_promedio:.1f} min)")
                    print(f"    → Agente extra liberado. Total: {self.agentes_activos}")

def simular_callcenter(usar_mara=False):
    env = simpy.Environment()
    callcenter = CallCenter(env, NUM_AGENTES_BASE, usar_mara)

    def proceso_llamada(env, llamada_id, callcenter):
        """Procesa una llamada individual"""
        hora_llegada = env.now

        with callcenter.agentes.request() as solicitud:
            # Esperar máximo TIEMPO_MAX_ESPERA minutos (convertido a horas)
            tiempo_max_espera_horas = TIEMPO_MAX_ESPERA / 60
            resultado = yield solicitud | env.timeout(tiempo_max_espera_horas)

            if solicitud in resultado:
                # Llamada atendida exitosamente
                tiempo_espera = (env.now - hora_llegada) * 60  # Convertir a minutos

                # Registrar en métricas
                if usar_mara:
                    metricas_con_mara["llamadas_atendidas"] += 1
                    metricas_con_mara["tiempo_espera_promedio"].append(tiempo_espera)
                    callcenter.cola_historico.append(tiempo_espera)

                    # Calcular satisfacción (inversa al tiempo de espera)
                    satisfaccion = max(1, 10 - (tiempo_espera / 3))
                    metricas_con_mara["satisfaccion_promedio"].append(satisfaccion)
                else:
                    metricas_sin_mara["llamadas_atendidas"] += 1
                    metricas_sin_mara["tiempo_espera_promedio"].append(tiempo_espera)

                    satisfaccion = max(1, 10 - (tiempo_espera / 3))
                    metricas_sin_mara["satisfaccion_promedio"].append(satisfaccion)

                # Atender la llamada (duración: 4-12 minutos)
                duracion_llamada = random.uniform(4/60, 12/60)  # Convertir a horas
                yield env.process(callcenter.atender_llamada(duracion_llamada))

            else:
                # Llamada abandonada por espera excesiva
                if usar_mara:
                    metricas_con_mara["llamadas_abandonadas"] += 1
                    callcenter.cola_historico.append(TIEMPO_MAX_ESPERA)  # Tiempo máximo alcanzado
                else:
                    metricas_sin_mara["llamadas_abandonadas"] += 1

    def generador_llamadas(env, callcenter):
        """Genera llamadas entrantes con patrones realistas"""
        llamada_id = 0

        while True:
            # Patrón de llegadas: más llamadas en horario laboral
            hora_del_dia = env.now % 24

            if 8 <= hora_del_dia <= 18:  # Horario pico
                tasa_llegada = random.uniform(0.5/60, 2/60)  # 0.5-2 minutos entre llamadas
            else:  # Horario valle
                tasa_llegada = random.uniform(2/60, 5/60)    # 2-5 minutos entre llamadas

            yield env.timeout(tasa_llegada)

            # Generar 1-3 llamadas simultáneas
            llamadas_simultaneas = random.randint(1, 3)
            for _ in range(llamadas_simultaneas):
                env.process(proceso_llamada(env, llamada_id, callcenter))
                llamada_id += 1

    # Iniciar procesos de simulación
    env.process(generador_llamadas(env, callcenter))
    if usar_mara:
        env.process(callcenter.modulo_mara())

    # Ejecutar simulación
    env.run(until=TIEMPO_SIMULACION)

# EJECUTAR SIMULACIONES
print("\n🔴 EJECUTANDO CALL CENTER TRADICIONAL...")
simular_callcenter(usar_mara=False)

print("\n🟢 EJECUTANDO CALL CENTER CON MARA...")
simular_callcenter(usar_mara=True)

# CALCULAR MÉTRICAS FINALES
def calcular_metricas_finales(metricas):
    """Calcula métricas resumen a partir de los datos recolectados"""
    if metricas["tiempo_espera_promedio"]:
        metricas["tiempo_espera_final"] = np.mean(metricas["tiempo_espera_promedio"])
    else:
        metricas["tiempo_espera_final"] = 0

    if metricas["satisfaccion_promedio"]:
        metricas["satisfaccion_final"] = np.mean(metricas["satisfaccion_promedio"])
    else:
        metricas["satisfaccion_final"] = 0

    total_llamadas = metricas["llamadas_atendidas"] + metricas["llamadas_abandonadas"]
    if total_llamadas > 0:
        metricas["tasa_abandono"] = (metricas["llamadas_abandonadas"] / total_llamadas) * 100
        metricas["eficiencia"] = (metricas["llamadas_atendidas"] / TIEMPO_SIMULACION) * 24  # Llamadas/día
    else:
        metricas["tasa_abandono"] = 0
        metricas["eficiencia"] = 0

    return metricas

# Aplicar cálculos
metricas_sin_mara = calcular_metricas_finales(metricas_sin_mara)
metricas_con_mara = calcular_metricas_finales(metricas_con_mara)

# MOSTRAR RESULTADOS
print("\n" + "="*70)
print("📊 RESULTADOS COMPARATIVOS - SERVICIO AL CLIENTE")
print("="*70)

resultados = pd.DataFrame({
    "Métrica": [
        "Llamadas Atendidas",
        "Llamadas Abandonadas",
        "Tasa de Abandono (%)",
        "Tiempo Espera Promedio (min)",
        "Satisfacción Cliente (1-10)",
        "Eficiencia (llamadas/día)",
        "Agentes Activos (promedio)",
        "Intervenciones MARA"
    ],
    "Sin MARA": [
        metricas_sin_mara["llamadas_atendidas"],
        metricas_sin_mara["llamadas_abandonadas"],
        f"{metricas_sin_mara['tasa_abandono']:.1f}%",
        f"{metricas_sin_mara['tiempo_espera_final']:.2f}",
        f"{metricas_sin_mara['satisfaccion_final']:.1f}",
        f"{metricas_sin_mara['eficiencia']:.1f}",
        f"{metricas_sin_mara['agentes_activos']}",
        "N/A"
    ],
    "Con MARA": [
        metricas_con_mara["llamadas_atendidas"],
        metricas_con_mara["llamadas_abandonadas"],
        f"{metricas_con_mara['tasa_abandono']:.1f}%",
        f"{metricas_con_mara['tiempo_espera_final']:.2f}",
        f"{metricas_con_mara['satisfaccion_final']:.1f}",
        f"{metricas_con_mara['eficiencia']:.1f}",
        f"{metricas_con_mara['agentes_activos']}",
        f"{metricas_con_mara['intervenciones_mara']}"
    ]
})

print(resultados)

# GRÁFICOS COMPARATIVOS
print("\n📈 GENERANDO GRÁFICOS COMPARATIVOS...")

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Gráfico 1: Llamadas atendidas vs abandonadas
categorias = ['Atendidas', 'Abandonadas']
sin_mara_vals = [metricas_sin_mara['llamadas_atendidas'], metricas_sin_mara['llamadas_abandonadas']]
con_mara_vals = [metricas_con_mara['llamadas_atendidas'], metricas_con_mara['llamadas_abandonadas']]

x = np.arange(len(categorias))
ax1.bar(x - 0.2, sin_mara_vals, 0.4, label='Sin MARA', color='red', alpha=0.7)
ax1.bar(x + 0.2, con_mara_vals, 0.4, label='Con MARA', color='green', alpha=0.7)
ax1.set_ylabel('Cantidad de Llamadas')
ax1.set_title('DISTRIBUCIÓN DE LLAMADAS')
ax1.set_xticks(x)
ax1.set_xticklabels(categorias)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfico 2: Métricas de calidad de servicio
metricas_calidad = ['Tiempo Espera (min)', 'Satisfacción', 'Tasa Abandono (%)']
sin_mara_calidad = [metricas_sin_mara['tiempo_espera_final'],
                   metricas_sin_mara['satisfaccion_final'],
                   metricas_sin_mara['tasa_abandono']]
con_mara_calidad = [metricas_con_mara['tiempo_espera_final'],
                   metricas_con_mara['satisfaccion_final'],
                   metricas_con_mara['tasa_abandono']]

x2 = np.arange(len(metricas_calidad))
ax2.bar(x2 - 0.2, sin_mara_calidad, 0.4, label='Sin MARA', color='red', alpha=0.7)
ax2.bar(x2 + 0.2, con_mara_calidad, 0.4, label='Con MARA', color='green', alpha=0.7)
ax2.set_ylabel('Valor')
ax2.set_title('MÉTRICAS DE CALIDAD DE SERVICIO')
ax2.set_xticks(x2)
ax2.set_xticklabels(metricas_calidad, rotation=45)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Gráfico 3: Eficiencia operativa
eficiencia_labels = ['Llamadas/Día', 'Agentes Activos']
sin_mara_ef = [metricas_sin_mara['eficiencia'], metricas_sin_mara['agentes_activos']]
con_mara_ef = [metricas_con_mara['eficiencia'], metricas_con_mara['agentes_activos']]

x3 = np.arange(len(eficiencia_labels))
ax3.bar(x3 - 0.2, sin_mara_ef, 0.4, label='Sin MARA', color='red', alpha=0.7)
ax3.bar(x3 + 0.2, con_mara_ef, 0.4, label='Con MARA', color='green', alpha=0.7)
ax3.set_ylabel('Valor')
ax3.set_title('EFICIENCIA OPERATIVA')
ax3.set_xticks(x3)
ax3.set_xticklabels(eficiencia_labels)
ax3.legend()
ax3.grid(True, alpha=0.3)

# Gráfico 4: Mejora porcentual
mejoras = [
    ((metricas_con_mara['llamadas_atendidas'] - metricas_sin_mara['llamadas_atendidas']) / metricas_sin_mara['llamadas_atendidas']) * 100,
    ((metricas_sin_mara['tasa_abandono'] - metricas_con_mara['tasa_abandono']) / metricas_sin_mara['tasa_abandono']) * 100,
    ((metricas_con_mara['satisfaccion_final'] - metricas_sin_mara['satisfaccion_final']) / metricas_sin_mara['satisfaccion_final']) * 100
]
mejora_labels = ['Llamadas Atendidas', 'Reducción Abandono', 'Mejora Satisfacción']

colors = ['green' if x > 0 else 'red' for x in mejoras]
ax4.bar(mejora_labels, mejoras, color=colors, alpha=0.7)
ax4.set_ylabel('Mejora (%)')
ax4.set_title('IMPACTO DE MARA - MEJORAS PORCENTUALES')
ax4.tick_params(axis='x', rotation=45)
ax4.grid(True, alpha=0.3)

# Añadir valores en las barras
for i, v in enumerate(mejoras):
    ax4.text(i, v + (1 if v > 0 else -1), f'{v:.1f}%',
             ha='center', va='bottom' if v > 0 else 'top', fontweight='bold')

plt.tight_layout()
plt.show()

# ANÁLISIS DE IMPACTO DETALLADO
print("\n" + "="*70)
print("📈 ANÁLISIS DETALLADO DE IMPACTO - MARA")
print("="*70)

mejora_atendidas = ((metricas_con_mara['llamadas_atendidas'] - metricas_sin_mara['llamadas_atendidas']) /
                    metricas_sin_mara['llamadas_atendidas']) * 100
reduccion_abandono = metricas_sin_mara['tasa_abandono'] - metricas_con_mara['tasa_abandono']
mejora_satisfaccion = metricas_con_mara['satisfaccion_final'] - metricas_sin_mara['satisfaccion_final']
reduccion_espera = metricas_sin_mara['tiempo_espera_final'] - metricas_con_mara['tiempo_espera_final']

print(f"🔹 EFECTIVIDAD OPERATIVA:")
print(f"   • Llamadas atendidas: +{mejora_atendidas:.1f}%")
print(f"   • Tasa de abandono: -{reduccion_abandono:.1f} puntos porcentuales")
print(f"   • Tiempo de espera: -{reduccion_espera:.2f} minutos (-{(reduccion_espera/metricas_sin_mara['tiempo_espera_final'])*100:.1f}%)")

print(f"\n🔹 CALIDAD DE SERVICIO:")
print(f"   • Satisfacción cliente: +{mejora_satisfaccion:.1f} puntos (+{(mejora_satisfaccion/metricas_sin_mara['satisfaccion_final'])*100:.1f}%)")
print(f"   • Eficiencia: {metricas_con_mara['eficiencia']:.1f} vs {metricas_sin_mara['eficiencia']:.1f} llamadas/día")

print(f"\n🔹 INTELIGENCIA OPERATIVA DE MARA:")
print(f"   • Agentes activos promedio: {metricas_con_mara['agentes_activos']} vs {metricas_sin_mara['agentes_activos']} fijos")
print(f"   • Intervenciones automáticas: {metricas_con_mara['intervenciones_mara']}")

# BENEFICIO ECONÓMICO ESTIMADO
print(f"\n💰 BENEFICIO ECONÓMICO ESTIMADO:")
valor_llamada = 25  # USD por llamada atendida
costo_agente_hora = 15  # USD por hora por agente

incremento_ingresos = (metricas_con_mara['llamadas_atendidas'] - metricas_sin_mara['llamadas_atendidas']) * valor_llamada
costo_agentes_extra = (metricas_con_mara['agentes_activos'] - metricas_sin_mara['agentes_activos']) * costo_agente_hora * 24 * 7
beneficio_neto = incremento_ingresos - costo_agentes_extra

print(f"   • Incremento ingresos: ${incremento_ingresos:.0f} USD/semana")
print(f"   • Costo agentes extra: ${costo_agentes_extra:.0f} USD/semana")
print(f"   • Beneficio neto: ${beneficio_neto:.0f} USD/semana")
print(f"   • Proyección anual: ${beneficio_neto * 52:,.0f} USD/año")

print(f"\n✅ CONCLUSIÓN: MARA demuestra una mejora significativa en todos los indicadores ")
print(f"   clave de servicio al cliente, con un impacto económico positivo sustancial.")